# Part A2 
This notebook containts the preprocessing and data generation steps that were performed to obtain the graph data.

First step: created csv files with default commands using the github recommended csv converter: XMLToCSV.py --annotate --neo4j data/dblp.xml data/dblp.dtd data/output.csv

## CSV processing

In [1]:
#import of required packages


import pandas as pd
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import random
import lipsum

### 0. Helper functions

In [2]:
#stopwords for stopword removal in keyword generation
STOP_WORDS = set(stopwords.words('english')) 

#collection of database community keywords
DATABASE_COMMUNITY_KEYWORDS = ["data management","indexing", "data modeling", "big data", "data processing", "data storage","data querying"]

def create_keywords(title, for_db_community=False):
    """
    This functions takes in a title string and returns keywords based on that title. This is done by splitting the title into tokens
    and removing the stopwords. Since the database community keywords are usually 2 words long, they are not included this way. To have 
    enough papers with database community keywords, 1 database community keyword is added randomly to ~30% of papers.
    """
    title = re.sub(r'[^\w\s]', "", title) #remove punctuation
    word_tokens = word_tokenize(title)
    keywords = set([w.lower() for w in word_tokens if not w.lower() in STOP_WORDS])

    for kw in DATABASE_COMMUNITY_KEYWORDS:
        if kw in title:
            keywords.add(kw)

    if for_db_community:
        keywords.add(random.sample(DATABASE_COMMUNITY_KEYWORDS,1)[0])

    #add random database community kw to approx 10% of papers
    if(random.randrange(0,10,1)<= 1):
        keywords.add(random.sample(DATABASE_COMMUNITY_KEYWORDS,1)[0])

    return "|".join(keywords)

def trim_title(title):
    """
    During data loading we discovered that there are very few papers that have an extremly long title string which causes problems during 
    data loading. Therefore the title character limit is checked and set to 300 characters
    """
    if len(title)>300:
        return title[:300]
    
    return title


def generate_affiliation(row):
    """
    Generates a random affiliation tuple.
    """
    affil = {
        1: "Company",
        2: "University"
    }

    return f"{affil.get(random.randint(1,2))} {random.randint(1,30)}"

def generate_publisher(publisher):
    """
    Generates a random publisher string.
    """
    return f"Publisher {random.randint(1,10)}"

In [3]:
def doi_or_na(ee):
    try:
        doi = ee.split("|")[0]
        #returns the text if it is a doi, na otherwise
        if("doi" in ee):
            return ee
        
    except:
        print("no str found")
    
    return pd.NA

### 1. Article Preprocessing

extracting relevant article information from the articles csv.

In [4]:
#headers to keep from the csv
KEEP_HEADERS_ARTICLE = [
    "article:ID",
    "author:string[]",
    "crossref:string",#volume-key
    "editor:string[]",
    "ee:string[]",
    "journal:string",#journal:name
    "key:string", #paper:key
    "number:string",#volume of the year
    "pages:string",
    "title:string", #paper:title
    "volume:string",#consecutive issue number
    "year:int", #volume:year,
    "publisher:string"
]

In [5]:
header = """article:ID;author:string[];author-aux:string;author-orcid:string[];booktitle:string;cdate:date;cdrom:string;cite:string[];cite-label:string[];crossref:string;editor:string[];editor-orcid:string[];ee:string[];ee-type:string[];i:string[];journal:string;key:string;mdate:date;month:string;note:string[];note-label:string;note-type:string[];number:string;pages:string;publisher:string;publnr:string;publtype:string;sub:string[];sup:string[];title:string;title-bibtex:string;tt:string[];url:string[];volume:string;year:int""".split(";")

#load data with header
articles = pd.read_csv("data/output_article.csv", nrows=100000, sep=";", names=header)
articles = articles[KEEP_HEADERS_ARTICLE]

#typing columns
articles["crossref:string"] = articles["crossref:string"].astype("string")
articles["title:string"] = articles["title:string"].astype("string")
articles["author:string[]"] = articles["author:string[]"].astype("string")
pd.to_numeric(articles["number:string"], errors="coerce", downcast="integer")
pd.to_numeric(articles["year:int"], downcast="integer")

#filter down to year > 1985
articles = articles.dropna(subset="year:int")
articles = articles[articles["year:int"] > 1985]
print(len(articles))

C:\Users\schmi\AppData\Local\Temp\ipykernel_3324\2962555990.py:4: DtypeWarning: Columns (6,7,9,10,19,24,26,27,33) have mixed types. Specify dtype option on import or set low_memory=False.
  articles = pd.read_csv("data/output_article.csv", nrows=100000, sep=";", names=header)


98652


In [6]:
#get doi
articles["doi"] = articles["ee:string[]"].apply(lambda x: doi_or_na(str(x)))

In [7]:
#make database community

database_journals = ['Decis. Support Syst.', 'J. Assoc. Inf. Sci. Technol.', 'Big Data Res.']
all_journals = ['Web Intell. Agent Syst.', 'Web Intell.' 'IEEE Trans. Hum. Mach. Syst.', 
                'Comput. Chem. Eng.', 'IEEE Control. Syst. Lett.', 'Inf. Knowl. Syst. Manag.', 'Internet Math.',
                'Int. J. Web Inf. Syst.', 'Manag. Sci.' 'Simul.', 'Rev. Iberoam. de Tecnol. del Aprendiz.', 'Dyn. Games Appl.',
                'Decis. Support Syst.', 'J. Assoc. Inf. Sci. Technol.', 'Big Data Res.',
                'Web Intell. Agent Syst.', 'Web Intell.', 'IEEE Trans. Hum. Mach. Syst.']


#filter journals
articles["keep"] = articles["journal:string"].apply(lambda x: x in all_journals)
articles["db_community"] = articles["journal:string"].apply(lambda x: x in database_journals)

articles = articles[articles["keep"]]
print(len(articles))

18030


In [8]:
#drop rows with no journal name
articles = articles.dropna(subset=["journal:string"])

#drop articles without author
articles = articles.dropna(subset=["author:string[]"])

#drop with no pages
articles = articles.dropna(subset=["pages:string"])

#ensure title lenth
articles = articles.dropna(subset=["title:string"])
articles["title:string"] = articles["title:string"].map(trim_title)

#drop rows with missing year
articles = articles.dropna(subset=["year:int"])

#assert volume 1 for missing volume number
articles["number:string"] = articles["number:string"].fillna(1) #set volume number to 1 for missing ones

#create volumekey
articles["volume_key"] = articles["journal:string"] + articles["year:int"].astype("string") + articles["number:string"].astype("string")

#fill missing crossref
articles["crossref:string"] = articles["crossref:string"].fillna(articles["volume_key"])

#create corresponding_author
articles["corresponding_author"] = articles["author:string[]"].apply(lambda x: x.split("|")[0])

In [9]:
#generate keywords
articles["keywords"] = articles.apply(lambda row: create_keywords(row["title:string"], row["db_community"]), axis=1)
articles["in_db_community"] = articles["keywords"].apply(lambda x: len(set(DATABASE_COMMUNITY_KEYWORDS).intersection(set(x.split("|"))))>0)

In [10]:
#generate publishers
articles["publisher"] = articles["publisher:string"].apply(generate_publisher)

In [11]:
articles[articles["in_db_community"]].head()
#print(len(articles))

,article:ID,author:string[],crossref:string,editor:string[],ee:string[],journal:string,key:string,number:string,pages:string,title:string,...,year:int,publisher:string,doi,keep,db_community,volume_key,corresponding_author,keywords,in_db_community,publisher
4630,32735,Frank McCarey|Mel Ó Cinnéide|Nicholas Kushmerick,Web Intell. Agent Syst.20081,NaN,https://doi.org/10.3233/WIA-2008-0129,Web Intell. Agent Syst.,journals/wias/McCareyCK08,1,59-81,Knowledge reuse for software reuse.,...,2008,NaN,https://doi.org/10.3233/WIA-2008-0129,True,False,Web Intell. Agent Syst.20081,Frank McCarey,reuse|big data|software|knowledge,True,Publisher 9
4635,32740,Larry M. Manevitz|Malik Yousef,Web Intell. Agent Syst.20042,NaN,http://content.iospress.com/articles/web-intel...,Web Intell. Agent Syst.,journals/wias/ManevitzY04,2,137-144,A web navigation system based on a neural netw...,...,2004,NaN,<NA>,True,False,Web Intell. Agent Syst.20042,Larry M. Manevitz,usermodel|documents|based|big data|system|trai...,True,Publisher 3
4642,32747,Chih-Ming Chen,Web Intell. Agent Syst.20041,NaN,http://content.iospress.com/articles/web-intel...,Web Intell. Agent Syst.,journals/wias/Chen04,1,21-38,Incremental personalized web page mining utili...,...,2004,NaN,<NA>,True,False,Web Intell. Agent Syst.20041,Chih-Ming Chen,personalized|hcmac|mining|page|data processing...,True,Publisher 4
4643,32748,Haralambos Mouratidis|Manuel Kolp|Paolo Giorgi...,Web Intell. Agent Syst.20101,NaN,https://doi.org/10.3233/WIA-2010-0182,Web Intell. Agent Syst.,journals/wias/MouratidisKGF10,1,99-122,An architectural description language for secu...,...,2010,NaN,https://doi.org/10.3233/WIA-2010-0182,True,False,Web Intell. Agent Syst.20101,Haralambos Mouratidis,description|language|indexing|architectural|sy...,True,Publisher 7
4653,32758,Lin Li|Rui Zhang|Wenfeng Zhao|Xiaojie Li|Xicha...,Web Intell.20221,NaN,https://doi.org/10.3233/WEB-210478,Web Intell.,journals/wias/HuZLYLZZL22,1,21-35,Conflict detection in Task Heterogeneous Infor...,...,2022,NaN,https://doi.org/10.3233/WEB-210478,True,False,Web Intell.20221,Lin Li,detection|information|task|heterogeneous|data ...,True,Publisher 3


In [12]:
#save to file
articles.to_csv("data/articles_preprocessed.csv", index=False)

### 2. Inproceedings preprocessing

In [13]:
KEEP_HEADER_INPROCEEDING = [
    "inproceedings:ID",
    "author:string[]",
    "booktitle:string",#conference/forum title
    "crossref:string[]",#proceeding key
    "editor:string[]",
    "ee:string[]",
    "key:string",#inproceedings key
    "number:string",
    "pages:string",
    "title:string",
    "volume:int",
    "year:int"#year_held
]

In [14]:
header = """inproceedings:ID;author:string[];author-aux:string[];author-orcid:string[];booktitle:string;cdrom:string[];cite:string[];cite-label:string[];crossref:string[];editor:string[];editor-orcid:string[];ee:string[];ee-type:string[];i:string[];key:string;mdate:date;month:string;note:string;note-type:string;number:string;pages:string;publtype:string;sub:string[];sup:string[];title:string;title-bibtex:string;tt:string;url:string;volume:int;year:int""".split(";")
inproceedings = pd.read_csv("data/output_inproceedings.csv", nrows=100000, sep=";", names=header)
inproceedings = inproceedings[KEEP_HEADER_INPROCEEDING]

#typing columns
inproceedings["crossref:string[]"] = inproceedings["crossref:string[]"].astype("string")
inproceedings["title:string"] = inproceedings["title:string"].astype("string")
pd.to_numeric(inproceedings["number:string"], errors="coerce", downcast="integer")
pd.to_numeric(inproceedings["year:int"], downcast="integer")
inproceedings["author:string[]"] = inproceedings["author:string[]"].astype("string")

#only keep > 1985
inproceedings = inproceedings.dropna(subset="year:int")

inproceedings = inproceedings.dropna(subset="booktitle:string")
inproceedings = inproceedings[inproceedings["year:int"] > 1985]
print(len(inproceedings))

C:\Users\schmi\AppData\Local\Temp\ipykernel_3324\199994687.py:2: DtypeWarning: Columns (5,12,16,17,21,22,25) have mixed types. Specify dtype option on import or set low_memory=False.
  inproceedings = pd.read_csv("data/output_inproceedings.csv", nrows=100000, sep=";", names=header)


99405


In [15]:
conference_names = inproceedings["booktitle:string"].unique()

print(len(conference_names))

for name in conference_names:
    if "database" in name.lower():
        print(name)

756
Advanced Database Systems
Databases in Telecommunications
ILPS Workshop: Constraints and Databases


In [16]:

database_conferences = list(conference_names[0:20])
database_conferences.extend(["ACML", "Advanced Database Systems", "Databases in Telecommunications", "ILPS Workshop: Constraints and Databases", "Text Understanding in LILOG", 'COLT'])
print(database_conferences)

['SWEE', 'Text Understanding in LILOG', 'Advanced Database Systems', 'Constraint Programming', 'NIPS Causality: Objectives and Assessment', 'AISTATS', 'COLT', 'Active Learning and Experimental Design @ AISTATS', 'WAPA', 'ACML', 'KDD Cup', 'FSDM', 'ICGI', 'ICML Unsupervised and Transfer Learning', 'MLSB', 'NIPS Mini-Symposium on Causality in Time Series', 'Yahoo! Learning to Rank Challenge', 'WCCI Causation and Prediction Challenge', 'Gaussian Processes in Practice', 'ICML On-line Trading of Exploration and Exploitation', 'ACML', 'Advanced Database Systems', 'Databases in Telecommunications', 'ILPS Workshop: Constraints and Databases', 'Text Understanding in LILOG', 'COLT']


In [17]:
#database_conferences = ["ACML", "Advanced Database Systems", "Databases in Telecommunications", "ILPS Workshop: Constraints and Databases", "Text Understanding in LILOG", 'COLT']
all_conferences = conference_names[0:50]

In [18]:
class Counter:
    COUNTER = 0

def _keep(conf_name):
    Counter.COUNTER = Counter.COUNTER + 1
    return conf_name in all_conferences or Counter.COUNTER <= 10000


inproceedings["keep"] = inproceedings["booktitle:string"].apply(lambda x: _keep(x))

inproceedings.head()
inproceedings["db_community"] = inproceedings["booktitle:string"].apply(lambda x: x in database_conferences)

In [19]:
#filter 
inproceedings = inproceedings[inproceedings["keep"]]
print(len(inproceedings[inproceedings["db_community"]]))

1019


In [20]:
inproceedings.head()
#print(inproceedings["booktitle:string"].unique())

,inproceedings:ID,author:string[],booktitle:string,crossref:string[],editor:string[],ee:string[],key:string,number:string,pages:string,title:string,volume:int,year:int,keep,db_community
0,555,Arnon Rosenthal,SWEE,conf/swee/1998,NaN,http://www.mitre.org/support/swee/rosenthal.html,www/org/mitre/future,NaN,NaN,The Future of Classic Data Administration: Obj...,NaN,1998,True,True
2,159865,Sven Lorenz|Toni Bollinger|Udo Pletat,Text Understanding in LILOG,journals/lncs/1991-546,NaN,https://doi.org/10.1007/3-540-54594-8_72,journals/lncs/BollingerLP91,NaN,402-427,The LILOG Inference Engine.,NaN,1991,True,True
3,159866,Geoffrey Simmons|Kai-Uwe Carstensen,Text Understanding in LILOG,journals/lncs/1991-546,NaN,https://doi.org/10.1007/3-540-54594-8_83,journals/lncs/CarstensenS91,NaN,632-644,Why a Hill Can't be a Valley: Representing Ges...,NaN,1991,True,True
4,159867,David W. Flater|Yelena Yesha,Advanced Database Systems,journals/lncs/1993-759,NaN,https://doi.org/10.1007/3-540-57507-3_13,journals/lncs/FlaterY93,NaN,259-276,Towards Flexible Distributed Information Retri...,NaN,1993,True,True
5,159868,Claus-Rainer Rollinger|Otthein Herzog,Text Understanding in LILOG,journals/lncs/1991-546,NaN,https://doi.org/10.1007/3-540-54594-8_46,journals/lncs/RollingerH91,NaN,3-13,Introducing LILOG.,NaN,1991,True,True


In [21]:
#crossref is required to get the information from proceedings - thus crossref na are dropped
inproceedings = inproceedings.dropna(subset=["crossref:string[]"])

#drop articles without author
inproceedings = inproceedings.dropna(subset=["author:string[]"])

#create corresponding_author
inproceedings["corresponding_author"] = inproceedings["author:string[]"].apply(lambda x: x.split("|")[0])

#we have enough elements so drop the ones without pages
inproceedings = inproceedings.dropna(subset=["pages:string"])

#create doi
inproceedings["doi"] = inproceedings["ee:string[]"].apply(doi_or_na)

#ensure title lenth
inproceedings = inproceedings.dropna(subset=["title:string"])
inproceedings["title:string"] = inproceedings["title:string"].map(trim_title)

no str found
no str found
no str found


In [22]:
print(len(inproceedings))

9981


In [23]:
#generate keywords
inproceedings["keywords"] = inproceedings.apply(lambda x: create_keywords(x["title:string"], x["db_community"]), axis = 1)
inproceedings["is_workshop"] = inproceedings["title:string"].apply(lambda x: "workshop" in x.lower())

In [24]:
print(len(inproceedings[inproceedings["is_workshop"]]))

49


In [25]:
inproceedings.head()

,inproceedings:ID,author:string[],booktitle:string,crossref:string[],editor:string[],ee:string[],key:string,number:string,pages:string,title:string,volume:int,year:int,keep,db_community,corresponding_author,doi,keywords,is_workshop
2,159865,Sven Lorenz|Toni Bollinger|Udo Pletat,Text Understanding in LILOG,journals/lncs/1991-546,NaN,https://doi.org/10.1007/3-540-54594-8_72,journals/lncs/BollingerLP91,NaN,402-427,The LILOG Inference Engine.,NaN,1991,True,True,Sven Lorenz,https://doi.org/10.1007/3-540-54594-8_72,data modeling|inference|engine|lilog,False
3,159866,Geoffrey Simmons|Kai-Uwe Carstensen,Text Understanding in LILOG,journals/lncs/1991-546,NaN,https://doi.org/10.1007/3-540-54594-8_83,journals/lncs/CarstensenS91,NaN,632-644,Why a Hill Can't be a Valley: Representing Ges...,NaN,1991,True,True,Geoffrey Simmons,https://doi.org/10.1007/3-540-54594-8_83,gestalt|object|valley|cant|data processing|rep...,False
4,159867,David W. Flater|Yelena Yesha,Advanced Database Systems,journals/lncs/1993-759,NaN,https://doi.org/10.1007/3-540-57507-3_13,journals/lncs/FlaterY93,NaN,259-276,Towards Flexible Distributed Information Retri...,NaN,1993,True,True,David W. Flater,https://doi.org/10.1007/3-540-57507-3_13,flexible|retrieval|information|data processing...,False
5,159868,Claus-Rainer Rollinger|Otthein Herzog,Text Understanding in LILOG,journals/lncs/1991-546,NaN,https://doi.org/10.1007/3-540-54594-8_46,journals/lncs/RollingerH91,NaN,3-13,Introducing LILOG.,NaN,1991,True,True,Claus-Rainer Rollinger,https://doi.org/10.1007/3-540-54594-8_46,lilog|data processing|introducing,False
6,159869,Bharat K. Bhargava|Jagannathan Srinivasan|Pras...,Advanced Database Systems,journals/lncs/1993-759,NaN,https://doi.org/10.1007/3-540-57507-3_5,journals/lncs/BhargavaJSD93,NaN,87-103,Transition From A Relation To Object Model Imp...,NaN,1993,True,True,Bharat K. Bhargava,https://doi.org/10.1007/3-540-57507-3_5,object|transition|model|big data|implementatio...,False


In [26]:
#save to file
inproceedings.to_csv("data/inproceedings_preprocessed.csv", index=False)

### 3. Proceedings csv Preprocessing

In [27]:
KEEP_HEADER_PROCEEDING = [
    "proceedings:ID",
    "booktitle:string",
    "editor:string[]",
    "ee:string[]",
    "key:string",
    "number:string",
    "title:string",
    "volume:string",
    "year:int"
]

In [28]:
header = """proceedings:ID;address:string;author:string[];booktitle:string;cite:string[];cite-label:string[];editor:string[];editor-orcid:string[];ee:string[];ee-type:string[];i:string;isbn:string[];isbn-type:string[];journal:string;key:string;mdate:date;note:string[];note-type:string;number:string;pages:string;publisher:string[];publisher-href:string;publtype:string;school:string;series:string[];series-href:string[];sub:string;sup:string[];title:string;url:string[];volume:string;year:int""".split(";")
proceedings = pd.read_csv("data/output_proceedings.csv", nrows=100000, sep=";", names=header)
proceedings = proceedings[KEEP_HEADER_PROCEEDING]

#typing columns
proceedings["title:string"] = proceedings["title:string"].astype("string")
pd.to_numeric(proceedings["number:string"], errors="coerce", downcast="integer")
pd.to_numeric(proceedings["year:int"], downcast="integer")

C:\Users\schmi\AppData\Local\Temp\ipykernel_3324\3556115800.py:2: DtypeWarning: Columns (1,2,4,5,10,12,13,18,19,22,23,26,27) have mixed types. Specify dtype option on import or set low_memory=False.
  proceedings = pd.read_csv("data/output_proceedings.csv", nrows=100000, sep=";", names=header)


0        1999
1        2015
2        2013
3        2014
4        2019
         ... 
53876    2013
53877    2007
53878    1999
53879    2017
53880    2010
Name: year:int, Length: 53881, dtype: int16

In [29]:
proceedings.head()

,proceedings:ID,booktitle:string,editor:string[],ee:string[],key:string,number:string,title:string,volume:string,year:int
0,461,MMB (Kurzvorträge),Dieter Baum|Norbert Th. Müller|Richard Rödler,NaN,tr/trier/MI99-17,NaN,"MMB '99, Messung, Modellierung und Bewertung v...",99-16,1999
1,2240,NaN,Amir Hossein Alavi|Amir Hossein Gandomi|Conor ...,https://doi.org/10.1007/978-3-319-20883-1,reference/genetic/2015,NaN,Handbook of Genetic Programming Applications,NaN,2015
2,13516,NaN,Ankur Agarwal|Borko Furht,https://doi.org/10.1007/978-1-4614-8495-0,reference/med/2013,NaN,Handbook of Medical and Healthcare Technologies,NaN,2013
3,103589,Trans. Computational Collective Intelligence,Ngoc Thanh Nguyen,https://doi.org/10.1007/978-3-662-44509-9,journals/tcci/2014-14,NaN,Transactions on Computational Collective Intel...,8615,2014
4,103594,NaN,Marcin Hernes|Ngoc Thanh Nguyen|Ryszard Kowalczyk,https://doi.org/10.1007/978-3-662-58611-2,journals/tcci/2019-32,NaN,Transactions on Computational Collective Intel...,11370,2019


In [30]:
#drop where title not present
proceedings = proceedings.dropna(subset=["title:string"])

In [31]:
#filter proceedings down to relevant subset from selected papers
conference_crossrefs = inproceedings["crossref:string[]"].unique()
proceedings["in_selected_ip_subset"] = proceedings["key:string"].apply(lambda x: True if x in conference_crossrefs else pd.NA)
proceedings = proceedings.dropna(subset=["in_selected_ip_subset"])
print(sum(proceedings["in_selected_ip_subset"]))

#create doi
proceedings["doi"] = proceedings["ee:string[]"].apply(doi_or_na)

849


In [32]:
# get edition where possible
def get_edition(volume):
    try:
        return volume.split("-")[1]
    except Exception:
        return None
    
proceedings["edition"] = proceedings["volume:string"].map(get_edition)
proceedings["edition"] = proceedings["edition"].fillna(proceedings["volume:string"])


In [33]:
proceedings.to_csv("data/proceedings_preprocessed.csv", index=False)

## Data Generation

To be able to reflect everything that is required as described in the Lab text, we have to generate some additional data

### 1. Create citations

In [34]:
#paper keys and years from articles
article_keys = articles[["key:string","year:int"]]
#print(article_keys)

ip_keys = inproceedings[["key:string","year:int"]]
#print(ip_keys)

## union both
cite_keys = pd.concat([article_keys, ip_keys])
#cite_keys["year:int"].dtype

In [35]:
def generate_citations(df, year):
    """
    Selects 5 to 15 random citations from papers that have been published earlier than the current paper.
    """
    no_citations = random.randint(5,15)

    df = df[df["year:int"] < year]

    try:
        citation_keys = df["key:string"].sample(no_citations, random_state=42).tolist()
    except ValueError:
        #for the oldest papers obviously no citations can be selected, thus they won't have citations.
        print(f"could not make sample for year {year}")
        citation_keys = []

    return citation_keys    

In [36]:
#create citations across the board
cite_keys["cites"] = cite_keys["year:int"].apply(lambda year: generate_citations(cite_keys, year))

#create citations to journal articles, so that impact factor can calculated for more years
cite_keys["cites_journal_specific"] = cite_keys["year:int"].apply(lambda year: generate_citations(article_keys, year))

could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for year 1986
could not make sample for ye

In [37]:
cite_keys.head()

,key:string,year:int,cites,cites_journal_specific
4628,journals/wias/BouchetS13,2013,"[journals/jmlr/GutmannH10, journals/jasis/Naya...","[journals/cce/SilvadM12, journals/cce/Rhinehar..."
4629,journals/wias/WalshLB12,2012,"[journals/jasis/Oppenheim07, journals/procedia...","[journals/cce/GeS11, journals/jasis/Yakel05, j..."
4630,journals/wias/McCareyCK08,2008,"[journals/entcs/Ketema05, journals/cce/Mahdipo...","[journals/dss/LeeLY96, journals/dss/Fernandez-..."
4631,journals/wias/KulkarniV22,2022,"[journals/cce/ChoiLR04, journals/jmlr/WangTMB0...","[journals/cce/AdiC13, journals/wias/CaoZL06, j..."
4632,journals/wias/LiangZ19,2019,"[journals/dss/GuoZK08, journals/dss/ChiuHLC12,...","[journals/cce/RahimiAPA15, journals/dss/Gregg0..."


In [38]:
#list concatenation before writing list to file
cite_keys["cites"] = cite_keys["cites"].apply(lambda x: "|".join(x))
cite_keys["cites_journal_specific"] = cite_keys["cites_journal_specific"].apply(lambda x: "|".join(x))

cite_keys.head()

,key:string,year:int,cites,cites_journal_specific
4628,journals/wias/BouchetS13,2013,journals/jmlr/GutmannH10|journals/jasis/Nayar0...,journals/cce/SilvadM12|journals/cce/RhinehartS...
4629,journals/wias/WalshLB12,2012,journals/jasis/Oppenheim07|journals/procedia/B...,journals/cce/GeS11|journals/jasis/Yakel05|jour...
4630,journals/wias/McCareyCK08,2008,journals/entcs/Ketema05|journals/cce/Mahdipoor...,journals/dss/LeeLY96|journals/dss/Fernandez-Me...
4631,journals/wias/KulkarniV22,2022,journals/cce/ChoiLR04|journals/jmlr/WangTMB09|...,journals/cce/AdiC13|journals/wias/CaoZL06|jour...
4632,journals/wias/LiangZ19,2019,journals/dss/GuoZK08|journals/dss/ChiuHLC12|jo...,journals/cce/RahimiAPA15|journals/dss/Gregg09|...


In [39]:
cite_keys.to_csv("data/citations.csv", index=False)

### 2. Create editor relationships

In [40]:
#make authors a list
articles["author_list"] = articles["author:string[]"].apply(lambda x: x.split("|"))

#
# expand authors to separate rows
editors = articles.explode("author_list")

#group by researcher and journal
editors = editors.groupby(["author_list", "journal:string"]).agg(publish_count=("author_list","count"))

In [41]:
editors = editors.sort_values("publish_count", ascending=False)
#get top 3 authors of each journal
top_publishers = editors.groupby(["journal:string"]).head(3)

In [42]:
top_publishers = top_publishers.rename(columns={"author_list": "editor", "journal:string": "journal"})
top_publishers.head(25)

,,publish_count
author_list,journal:string,
Ignacio E. Grossmann,Comput. Chem. Eng.,153
Mike Thelwall,J. Assoc. Inf. Sci. Technol.,88
Loet Leydesdorff,J. Assoc. Inf. Sci. Technol.,83
Efstratios N. Pistikopoulos,Comput. Chem. Eng.,71
Rafiqul Gani,Comput. Chem. Eng.,68
Lutz Bornmann,J. Assoc. Inf. Sci. Technol.,57
Hsinchun Chen,Decis. Support Syst.,49
H. Raghav Rao,Decis. Support Syst.,33
Selwyn Piramuthu,Decis. Support Syst.,31


In [43]:
top_publishers.to_csv("data/editors.csv")

### 3. Create Conference Chairpersons Relationships

In [44]:
#make authors a list
inproceedings["author_list"] = inproceedings["author:string[]"].apply(lambda x: x.split("|"))

#
# expand authors to separate rows
editors = inproceedings.explode("author_list")

#group by researcher and journal
editors = editors.groupby(["author_list", "crossref:string[]"]).agg(publish_count=("author_list","count"))

In [45]:
editors = editors.sort_values("publish_count", ascending=False)
#get top 3 authors of each journal
top_publishers = editors.groupby(["crossref:string[]"]).head(3)

In [46]:
top_publishers = top_publishers.rename(columns={"author_list": "chair", "crossref:string[]": "conference_edition"})
top_publishers.sort_values("crossref:string[]").head(10)

,,publish_count
author_list,crossref:string[],
Mehmet Aksit,conf/abmb/2005,1
Perla Velasco Elizondo,conf/abmb/2005,1
Pablo Amaya,conf/abmb/2005,1
Niels Joncheere,conf/abmb/2006,1
Mehmet Aksit,conf/abmb/2006,1
Mathieu Braem,conf/abmb/2006,1
Hartmut Ehrig,conf/accat/2007,2
Ulrike Prange,conf/accat/2007,3
Leen Lambers,conf/accat/2007,1


In [47]:
top_publishers.to_csv("data/conference_chairs.csv")

### 4. Create review information 

First the no of reviewers per journal/conference is determined. Usually this will be 3, however there is some variation introduced.

In [48]:
journal_list = articles[["journal:string"]].drop_duplicates()
conference_list = inproceedings[["booktitle:string"]].drop_duplicates()

conference_list.columns = journal_list.columns

journal_conference_list = pd.concat([journal_list, conference_list])


journal_conference_list["no_reviewers"] = journal_conference_list["journal:string"].apply(lambda x: random.randint(1,4) if random.randint(1,10) < 4 else 3)
journal_conference_list.head(10)

,journal:string,no_reviewers
4628,Web Intell. Agent Syst.,3
4631,Web Intell.,3
5094,IEEE Trans. Hum. Mach. Syst.,3
5889,Comput. Chem. Eng.,1
11034,IEEE Control. Syst. Lett.,3
12786,Inf. Knowl. Syst. Manag.,3
13011,Internet Math.,3
13266,Int. J. Web Inf. Syst.,3
20840,Rev. Iberoam. de Tecnol. del Aprendiz.,3
21405,Dyn. Games Appl.,3


In [49]:
def get_reviewer_count(journal_name):
    return journal_conference_list[journal_conference_list["journal:string"] == journal_name]["no_reviewers"].values[0]

Then the Reviews are created. for a paper the researchers participating in that review are determined

In [50]:
art_short = articles[["key:string","journal:string", "author_list"]]
ip_short = inproceedings[["key:string","booktitle:string", "author_list"]]

ip_short.columns = art_short.columns

In [51]:
article_authors = pd.concat([ip_short, art_short])
article_authors.head(10)

,key:string,journal:string,author_list
2,journals/lncs/BollingerLP91,Text Understanding in LILOG,"[Sven Lorenz, Toni Bollinger, Udo Pletat]"
3,journals/lncs/CarstensenS91,Text Understanding in LILOG,"[Geoffrey Simmons, Kai-Uwe Carstensen]"
4,journals/lncs/FlaterY93,Advanced Database Systems,"[David W. Flater, Yelena Yesha]"
5,journals/lncs/RollingerH91,Text Understanding in LILOG,"[Claus-Rainer Rollinger, Otthein Herzog]"
6,journals/lncs/BhargavaJSD93,Advanced Database Systems,"[Bharat K. Bhargava, Jagannathan Srinivasan, P..."
7,journals/lncs/DorreR91,Text Understanding in LILOG,"[Ingo Raasch, Jochen Dörre]"
8,journals/lncs/Emde91,Text Understanding in LILOG,[Werner Emde]
9,journals/lncs/Ridoux94,Constraint Programming,[Olivier Ridoux]
10,journals/lncs/Blasius91,Text Understanding in LILOG,[Karl-Hans Bläsius]
11,journals/lncs/LuckP91,Text Understanding in LILOG,"[Kai von Luck, Thomas Pirlein]"


In [52]:
affiliated_authors = article_authors.groupby('journal:string').agg({'author_list': "sum"})
affiliated_authors["author_list"] = affiliated_authors["author_list"].apply(lambda x: set(x))

In [53]:
affiliated_authors_dict = affiliated_authors.to_dict("index")
all_authors = set()

for key, v in affiliated_authors_dict.items():
    all_authors = all_authors.union(v["author_list"])

print(len(all_authors))

47149


In [54]:
class Key:
    PREVIOUS_KEY = "Comput. Chem. Eng."

def get_reviewers(key):
    try:
        author_sample = random.sample(affiliated_authors_dict[key]["author_list"], 5)
        Key.PREVIOUS_KEY = key
    except ValueError:
        author_sample = random.sample(affiliated_authors_dict[Key.PREVIOUS_KEY]["author_list"], 5)
    
    return set(author_sample)

In [55]:
#get sample of affiliated reviewers
article_authors["reviewers"] = article_authors["journal:string"].apply(lambda x: get_reviewers(x))
article_authors.head(3)

C:\Users\schmi\AppData\Local\Temp\ipykernel_3324\1130410631.py:6: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  author_sample = random.sample(affiliated_authors_dict[key]["author_list"], 5)
C:\Users\schmi\AppData\Local\Temp\ipykernel_3324\1130410631.py:9: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  author_sample = random.sample(affiliated_authors_dict[Key.PREVIOUS_KEY]["author_list"], 5)


,key:string,journal:string,author_list,reviewers
2,journals/lncs/BollingerLP91,Text Understanding in LILOG,"[Sven Lorenz, Toni Bollinger, Udo Pletat]","{Albert Maier, Peter Gerstl, Tibor Kiss, Gert ..."
3,journals/lncs/CarstensenS91,Text Understanding in LILOG,"[Geoffrey Simmons, Kai-Uwe Carstensen]","{Peter H. Schmitt, Birgit Wesche, Stefan Benzs..."
4,journals/lncs/FlaterY93,Advanced Database Systems,"[David W. Flater, Yelena Yesha]","{Ramez Elmasri, Shashi K. Gadia, Narain H. Geh..."


In [56]:
#Remove paper authors from reviewers
article_authors["reviewers"] = article_authors.apply(lambda row: row["reviewers"].difference(set(row["author_list"])), axis=1)

In [57]:
#create random reviewers where not enough
def make_reviewers(current_reviewers:set, authors:set, journal:str):
    no_reviewers = get_reviewer_count(journal)

    while len(current_reviewers) < no_reviewers:
        #add new sample
        current_reviewers = current_reviewers.union(set(random.sample(all_authors, 1)))
        #remove current authors
        current_reviewers = current_reviewers.difference(authors)

    if len(current_reviewers) >no_reviewers:
        current_reviewers = set(random.sample(current_reviewers, no_reviewers))

    return current_reviewers

In [58]:
article_authors["reviewers"] = article_authors.apply(lambda row: make_reviewers(row["reviewers"], set(row["author_list"]), row["journal:string"]), axis=1)

C:\Users\schmi\AppData\Local\Temp\ipykernel_3324\2002427380.py:12: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  current_reviewers = set(random.sample(current_reviewers, no_reviewers))
C:\Users\schmi\AppData\Local\Temp\ipykernel_3324\2002427380.py:7: DeprecationWarning: Sampling from a set deprecated
since Python 3.9 and will be removed in a subsequent version.
  current_reviewers = current_reviewers.union(set(random.sample(all_authors, 1)))


In [59]:
reviewers = article_authors[["key:string", "reviewers"]].explode("reviewers")
reviewers.head()

,key:string,reviewers
2,journals/lncs/BollingerLP91,Roland Seiffert
2,journals/lncs/BollingerLP91,Albert Maier
2,journals/lncs/BollingerLP91,Peter Gerstl
3,journals/lncs/CarstensenS91,Stefan Benzschawel
3,journals/lncs/CarstensenS91,Simone Pribbenow


In [60]:
#generate review text
reviewers["review_text"] = reviewers["reviewers"].apply(lambda x: lipsum.generate_words(30))
reviewers["suggested_decision"] = reviewers["review_text"].apply(lambda x: random.sample(["acceptance", "conditional acceptance", "conditional rejection", "outright rejection"],1)[0] if random.randint(1,10) < 2 else "acceptance")

reviewers["suggested_decision"].unique()

array(['acceptance', 'conditional acceptance', 'outright rejection',
       'conditional rejection'], dtype=object)

In [61]:
reviewers["supports_acceptance"] = reviewers["suggested_decision"].apply(lambda x : x in ["acceptance", "conditional acceptance"])

In [62]:
acceptance_statistic = reviewers.groupby(["key:string"]).agg(acceptance_count=("supports_acceptance","sum"), no_reviewers=("reviewers", "count"))
acceptance_statistic["accepted"] = acceptance_statistic["acceptance_count"] > acceptance_statistic["no_reviewers"]/2
#acceptance_statistic[acceptance_statistic["accepted"] == False]
len(reviewers)

75040

In [63]:
reviewers.to_csv("data/reviewers.csv", index=False)

### Affiliations

In [64]:
affiliations = pd.concat([articles[["author_list"]], inproceedings[["author_list"]]])
affiliations = affiliations.explode("author_list")
affiliations["affiliation"] = affiliations["author_list"].apply(generate_affiliation)
affiliations["organization_type"] = affiliations["affiliation"].apply(lambda x: x.split(" ")[0])
affiliations.head()

,author_list,affiliation,organization_type
4628,François Bouchet,University 25,University
4628,Jean-Paul Sansonnet,Company 13,Company
4629,Alexander Borgida,Company 22,Company
4629,Michael L. Littman,Company 23,Company
4629,Thomas J. Walsh 0001,Company 10,Company


In [65]:
affiliations.to_csv("data/affiliations.csv", index=False)